# Llama 3.2 3B + Behavioral Cloning

*Contributed by [Emaad Manzoor](https://emaadmanzoor.com/) (emaadmanzoor@cornell.edu) to the [stopping-agents](https://github.com/emaadmanzoor/stopping-agents) repository.*

## Imports

Before running this notebook locally, you need to [install PyTorch](https://pytorch.org/get-started/locally/) for your hardware.

Then, you need to install the following packages:

   * transformers
   * datasets
   * accelerate
   * pandas
   * huggingface_hub (needed for Llama models)
   * scikit-learn
   * numpy

You an also use the `requirements.txt` in the [stopping-agents](https://github.com/emaadmanzoor/stopping-agents) repository.

In [ ]:
import datasets
import huggingface_hub # needed for Llama models
import math
import numpy as np
import pandas as pd
import torch
import transformers

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from tqdm.auto import tqdm

HF_TOKEN = "HF_TOKEN"

COST_PER_UNIT_TIME = 0.1
BENEFIT_PER_POSITIVE_OUTCOME = 10.0
DECISION_OPPORTUNITIES = [45, 60] # time in seconds at which the 
                                  # agent can decide to quit or wait
                                  # code is tailored to just 2 right now

## Load and process conversation data

We load a dataset of synthetic conversations available
in the `datasets` folder at [https://github.com/emaadmanzoor/stopping-agents/](https://github.com/emaadmanzoor/stopping-agents/). This example dataset is formatted in the PyAnnote diarized conversation format.

In [2]:
dataset_url = "https://raw.githubusercontent.com/emaadmanzoor/stopping-agents/refs/heads/main/datasets/synthetic_sales_conversations.csv?token=GHSAT0AAAAAADBUAD4WOA6XRF2GSIX5UC4Y2EEF66Q"

diarized_conversations = pd.read_csv(dataset_url)

diarized_conversations["is_sale"] =\
        diarized_conversations["outcome"].apply(
            lambda x: 1 if x == "sale" else 0 if x == "no sale" else np.nan)

diarized_conversations["duration"] =\
    diarized_conversations.groupby("conversation_id")["end_time"].transform("max")

diarized_conversations.head()

,conversation_id,speaker_id,start_time,end_time,text,outcome,is_sale,duration
0,20756_1,0,1.25,6.03,"Hello, is this Mr. Harris? My name is Leah fro...",no sale,0,62.07
1,20756_1,1,6.36,7.59,"Yes, speaking. I’m alright, thanks. Can I ask ...",no sale,0,62.07
2,20756_1,0,7.98,12.84,"Of course, thanks for asking. I’m reaching out...",no sale,0,62.07
3,20756_1,1,13.14,15.50,Alright… I guess I can listen for a minute.,no sale,0,62.07
4,20756_1,0,15.89,22.14,"Thank you! So, our new BrightSaver plan locks ...",no sale,0,62.07


### Split into train, validation, and test conversations

In [3]:
all_conversation_ids =\
    diarized_conversations[["conversation_id", "is_sale"]].drop_duplicates()["conversation_id"]\
        .values
all_outcomes =\
    diarized_conversations[["conversation_id", "is_sale"]].drop_duplicates()["is_sale"].values
    
train_conversation_ids, test_conversation_ids, train_outcomes, test_outcomes =\
    train_test_split(all_conversation_ids, all_outcomes, test_size=0.25, random_state=42,
                     stratify=all_outcomes)
train_conversation_ids, val_conversation_ids, train_outcomes, val_outcomes =\
    train_test_split(train_conversation_ids, train_outcomes, test_size=0.25, random_state=42,
                     stratify=train_outcomes)

diarized_conversations_train =\
    diarized_conversations[diarized_conversations["conversation_id"].isin(train_conversation_ids)]
diarized_conversations_val =\
    diarized_conversations[diarized_conversations["conversation_id"].isin(val_conversation_ids)]
diarized_conversations_test =\
    diarized_conversations[diarized_conversations["conversation_id"].isin(test_conversation_ids)]

print(len(diarized_conversations_train), "train conversations.")
print(len(diarized_conversations_val), "validation conversations.")
print(len(diarized_conversations_test), "test conversations.")

1903 train conversations.
651 validation conversations.
860 test conversations.


### Accumulate transcripts at each decision opportunity

In [4]:
m1, m2 = sorted(DECISION_OPPORTUNITIES)

data_transcripts = {}
for df, dftype in zip([diarized_conversations_train,
                       diarized_conversations_val,
                       diarized_conversations_test],
                      ["train", "val", "test"]):
    
    data_transcripts[dftype] = df.copy()

    data_transcripts[dftype]["transcript"] =\
        "Speaker " +\
        data_transcripts[dftype]["speaker_id"].astype(str) + ": " +\
        data_transcripts[dftype]["text"]

    transcripts = {}
    for m in [m1, m2]: 
        transcripts[m] =\
            data_transcripts[dftype].loc[(data_transcripts[dftype]["end_time"]>=0) &
                                         (data_transcripts[dftype]["end_time"]<m)]\
                    .groupby("conversation_id")["transcript"]\
                    .apply(lambda x: '\n'.join(x))\
                    .reset_index(name="transcript_speaker_" + str(m))

    data_transcripts[dftype] = \
        pd.merge(data_transcripts[dftype][["conversation_id",
                                   "duration",
                                   "is_sale"]].drop_duplicates(),
                 transcripts[m1],
                 on="conversation_id", how="left", validate="one_to_one")\
                    .merge(transcripts[m2],
                           on="conversation_id", how="left", validate="one_to_one")

print(len(data_transcripts["train"]), "train conversations.")
print(len(data_transcripts["val"]), "validation conversations.")
print(len(data_transcripts["test"]), "test conversations.")

assert len(data_transcripts["train"]) +\
         len(data_transcripts["val"]) +\
         len(data_transcripts["test"]) == diarized_conversations["conversation_id"].nunique()

data_transcripts["train"].head()

112 train conversations.
38 validation conversations.
50 test conversations.


,conversation_id,duration,is_sale,transcript_speaker_45,transcript_speaker_60
0,20756_1,62.07,0,"Speaker 0: Hello, is this Mr. Harris? My name ...","Speaker 0: Hello, is this Mr. Harris? My name ..."
1,59321_6,55.37,0,Speaker 0: Good afternoon! Is this Ms. Parker?...,Speaker 0: Good afternoon! Is this Ms. Parker?...
2,92837_7,43.64,0,"Speaker 0: Good afternoon, may I speak with Mr...","Speaker 0: Good afternoon, may I speak with Mr..."
3,58241_9,50.01,0,"Speaker 0: Hello, may I speak with Ms. Jenkins...","Speaker 0: Hello, may I speak with Ms. Jenkins..."
4,20567_11,45.19,0,"Speaker 0: Good afternoon, is this Mr. Carver?...","Speaker 0: Good afternoon, is this Mr. Carver?..."


### Construct states

Wrap the transcript until each decision opportunity in a prompt to construct the state.

In [5]:
def convert_to_state(transcript, t):
    assert type(transcript) == str

    state = "Below is the first " + str(t) +\
            " seconds of the sales call between the sales agent Speaker 0 and" +\
            " the customer Speaker 1:\n" +\
            transcript + "\n" +\
            "Will this call end in a sale (respond with 'yes' or 'no'):  "

    return state

for df in [data_transcripts["train"],
           data_transcripts["val"],
           data_transcripts["test"]]:

    for m in [m1, m2]:
        df.loc[:, "s" + str(m)] = df.apply(lambda x:
                                            convert_to_state(x["transcript_speaker_" + str(m)],
                                                             m), axis=1)

print("Example state at", m1, "seconds:")
print(data_transcripts["train"]["s" + str(m1)].values[0])
print()
print("Example state at", m2, "seconds:")
print(data_transcripts["train"]["s" + str(m2)].values[0])

Example state at 45 seconds:
Below is the first 45 seconds of the sales call between the sales agent Speaker 0 and the customer Speaker 1:
Speaker 0: Hello, is this Mr. Harris? My name is Leah from Sunview Energy—how are you today?
Speaker 1: Yes, speaking. I’m alright, thanks. Can I ask what this is about?
Speaker 0: Of course, thanks for asking. I’m reaching out because we’re offering a new energy plan that could qualify you for a 15% discount on your electric bill. I wanted to see if I could quickly tell you about it.
Speaker 1: Alright… I guess I can listen for a minute.
Speaker 0: Thank you! So, our new BrightSaver plan locks in your rate for twelve months—there’s no change in price based on the time of day, and there are no hidden fees. And for this month, you’d also get an automatic 15% off your supply charges.
Speaker 1: Is this something I have to switch providers for? I’m pretty happy with who I have now.
Speaker 0: You would stay connected to your local utility for service a

### Construct optimal state-action pairs

This is the most important part of the behavioral cloning dataset construction process. Given the `COST_PER_UNIT_TIME` and `BENEFIT_PER_POSITIVE_OUTCOME`, the code below maps the conversation transcript before each decision opportunity to the optimal action (`wait` or `quit`) to take in that state. The optimal action is the one that maximizes the cumulative reward.

In [6]:
optimal_state_action_pairs = {}
for dftype, df in data_transcripts.items():
    df["rq" + str(m1)] = -m1 * COST_PER_UNIT_TIME # stop at 30

    # continue at 30, stop at 60
    df["rq" + str(m2)] = df["is_sale"].astype(int)\
                        * BENEFIT_PER_POSITIVE_OUTCOME\
                        * (df["duration"]<=m2).astype(int) \
                        - df["duration"].apply(lambda x: min(m2, x)) * COST_PER_UNIT_TIME
    
    # continue at 30, continue at 60, continue at 90 = never quit                
    df["rc" + str(m2)] = df["is_sale"].astype(int) * BENEFIT_PER_POSITIVE_OUTCOME\
                        - df["duration"] * COST_PER_UNIT_TIME

    df["max_reward"] = df[["rq" + str(m1), "rq" + str(m2), "rc" + str(m2)]].max(axis=1)

    # optimal to quit at 30
    df.loc[df["max_reward"]==df["rq" + str(m1)], "a" + str(m1)] = "no"
    df.loc[df["max_reward"]==df["rq" + str(m1)], "a" + str(m2)] = "no"

    # optimal to continue at 30, stop at 60
    df.loc[df["max_reward"]==df["rq" + str(m2)], "a"  + str(m1)] = "yes"
    df.loc[df["max_reward"]==df["rq" + str(m2)], "a"  + str(m2)] = "no"

    # optimal to continue at 30, continue at 60, continue at 90
    df.loc[df["max_reward"]==df["rc" + str(m2)], "a" + str(m1)] = "yes"
    df.loc[df["max_reward"]==df["rc" + str(m2)], "a" + str(m2)] = "yes"

    optimal_state_action_pairs[dftype] = []
    
    for m in [m1, m2]:
        optimal_actions = df[df["duration"]>=m]\
                                [["conversation_id", "s" + str(m), "a" + str(m), "is_sale", 
                                  "duration"]]
        optimal_actions = optimal_actions.values.tolist()
        optimal_state_action_pairs[dftype].extend(optimal_actions)

    optimal_state_action_pairs[dftype] =\
        pd.DataFrame(optimal_state_action_pairs[dftype],
                     columns=["conversation_id", "state", "action",
                              "is_sale", "duration"])

print("Distribution of actions in the optimal state-action pairs:")
print(optimal_state_action_pairs["train"]["action"].value_counts())
print(optimal_state_action_pairs["val"]["action"].value_counts())
print(optimal_state_action_pairs["test"]["action"].value_counts())
if len(optimal_state_action_pairs["train"]["action"].value_counts()) == 1:
    print("Only one optimal action in the training set; imitation learning will fail.")

optimal_state_action_pairs["train"].head()

Distribution of actions in the optimal state-action pairs:
action
yes    108
no      67
Name: count, dtype: int64
action
yes    38
no     22
Name: count, dtype: int64
action
yes    49
no     30
Name: count, dtype: int64


,conversation_id,state,action,is_sale,duration
0,20756_1,Below is the first 45 seconds of the sales cal...,no,0,62.07
1,59321_6,Below is the first 45 seconds of the sales cal...,no,0,55.37
2,58241_9,Below is the first 45 seconds of the sales cal...,no,0,50.01
3,20567_11,Below is the first 45 seconds of the sales cal...,no,0,45.19
4,10523_13,Below is the first 45 seconds of the sales cal...,no,0,65.04


## Behavioral Cloning

Now we can fine-tune our large language model to generate the optimal action for every state.

We fine-tune using `Trainer` in `Transformers` library instead of `SFTTRainer` in `TRL` for 2 reasons:

   1. By default, `SFTTrainer` calculates the loss for *all* tokens, including ones in the prompt. One way to work around this using the prompt-completion pairs data format.

   2. `SFTTrainer` does not enable custom metrics. We want to report the action predicton validation AUC during training, since it is more correlated with our final goal than the validation loss.

### Load base model

In [7]:
huggingface_hub.login(token=HF_TOKEN)

model_name = "meta-llama/Llama-3.2-3B" # base model, not instruction-tuned
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = transformers.AutoModelForCausalLM.from_pretrained(model_name, torch_dtype="auto")
tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.padding_side = 'left'

if len(tokenizer) > model.get_input_embeddings().weight.shape[0]:
    print("WARNING: Resizing the embedding matrix to match the tokenizer vocab size.")
    model.resize_token_embeddings(len(tokenizer))

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

### Construct and tokenize fine-tuning datasets

We perform manual masking, so the loss is only calculated for the generated actions.

In [8]:
train_dataset = datasets.Dataset.from_dict(
    {"prompt": [state for state in optimal_state_action_pairs["train"]["state"].values], 
     "completion": [action.strip()
                    for action in optimal_state_action_pairs["train"]["action"].values]}).shuffle()
    
val_dataset = datasets.Dataset.from_dict(
    {"prompt": [state for state in optimal_state_action_pairs["val"]["state"].values],
     "completion": [action.strip()
                    for action in optimal_state_action_pairs["val"]["action"].values]}).shuffle()

def tokenize_fn(example, add_label):
    # start with the BOS token if it exists
    if tokenizer.bos_token is not None:
        encoded_prompt = tokenizer.encode(tokenizer.bos_token +
                                          example["prompt"],              
                                          add_special_tokens=False)
    else:
        encoded_prompt = tokenizer.encode(example["prompt"], 
                                          add_special_tokens=False)

    # add the label if needed for the training and validation datasets
    if add_label:
        encoded_label = tokenizer.encode(example["completion"] + tokenizer.eos_token, 
                                         add_special_tokens=False)
        return {"input_ids": encoded_prompt + encoded_label,
                "attention_mask" : [1] * (len(encoded_prompt) + len(encoded_label)),
                "labels": [-100] * len(encoded_prompt) + encoded_label}
    else:
        return {"input_ids": encoded_prompt,
                "attention_mask": [1] * len(encoded_prompt),
                "labels": [-100] * len(encoded_prompt)}

train_dataset = train_dataset.map(tokenize_fn,
                                  remove_columns=["prompt", "completion"], 
                                  fn_kwargs={"add_label": True})
val_dataset = val_dataset.map(tokenize_fn, 
                              remove_columns=["prompt", "completion"], 
                              fn_kwargs={"add_label": True})

print("Tokenization test:")
print(tokenizer.decode(train_dataset[0]["input_ids"]))
print()
print("Expected Label (action):")
print(tokenizer.decode([l for l in train_dataset[0]["labels"] if l!=-100]))

Map:   0%|          | 0/175 [00:00<?, ? examples/s]

Map:   0%|          | 0/60 [00:00<?, ? examples/s]

Tokenization test:
<|begin_of_text|>Below is the first 45 seconds of the sales call between the sales agent Speaker 0 and the customer Speaker 1:
Speaker 0: Good afternoon, this is Marcus from Greenwave Energy. Am I speaking with Ms. Lopez?
Speaker 1: Hi, yes, this is her.
Speaker 0: Fantastic! I’ll keep this brief. We have a new energy plan with a guaranteed rate and monthly discounts for loyal clients. Are you open to hearing a quick summary?
Speaker 1: Alright, sure. Go ahead.
Speaker 0: Thank you! With the Greenwave Saver Plan, you lock in a fixed rate on electricity for a year. We're offering a $7 discount each month on your bill and a one-time $30 sign-up bonus. All this, with no contract lock-in or exit fees.
Speaker 1: Hmm. What's the rate compared to what I'm paying now?
Speaker 0: Great question. On your most recent bill, you were charged $0.15 per kWh. Our plan offers $0.132 per kWh, so you'd see a savings, plus the ongoing monthly discount.
Speaker 1: I don’t know… It sound

### Fine-tune

In [9]:
YES_TOKEN_ID = tokenizer.encode("yes", add_special_tokens=False)[-1]
NO_TOKEN_ID = tokenizer.encode("no", add_special_tokens=False)[-1]

def preprocess_logits_for_metrics(logits, labels):
    """
    Original Trainer may have a memory leak. 
    This is a workaround to avoid storing too many tensors that are not needed.
    """
    return logits[:, -3, :] # last non-padding token logits only, for causal LM

def compute_metrics(eval_preds):
    logits, labels = eval_preds
    labels = [l[(l != -100) & (l != tokenizer.pad_token_id)][0] for l in labels]

    logprobs = [torch.log_softmax(torch.from_numpy(s), dim=-1).numpy()
                for s in logits]
    labelprobs = [math.exp(logprob[label]) for logprob, label in zip(logprobs, labels)]

    ytrue = [1 if label == YES_TOKEN_ID else 0 for label in labels]
    ypred = [labelprob if label==YES_TOKEN_ID else 1.0 - labelprob
             for labelprob, label in zip(labelprobs, labels)]
    auc = roc_auc_score(ytrue, ypred)

    return {"auc": auc}

training_args = transformers.TrainingArguments(
    output_dir="./llama-3.2-3B/",
    overwrite_output_dir=True,
    remove_unused_columns=False,

    save_strategy="best",
    logging_strategy="epoch",
    eval_strategy="epoch",
    save_total_limit=1,

    # on 1 H100 96GB: batch size of 12-16 for 3B works
    per_device_train_batch_size=12,
    per_device_eval_batch_size=12,
    gradient_accumulation_steps=1,

    num_train_epochs=10,
    learning_rate=1e-4,
    optim="adamw_torch",

    load_best_model_at_end=True,
    metric_for_best_model="auc",
    greater_is_better=True,

    report_to="none", # change to wandb if needed
    save_safetensors=False, # needed to load saved models

    # change to suit hardware
    bf16=True, 
    fp16=False,
)

trainer = transformers.Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset.shuffle(),
    eval_dataset=val_dataset,
    processing_class=tokenizer,
    data_collator=transformers.data.DataCollatorForSeq2Seq(tokenizer),
    compute_metrics=compute_metrics,
    preprocess_logits_for_metrics=preprocess_logits_for_metrics,
    callbacks=[transformers.EarlyStoppingCallback(early_stopping_patience=1)]
)

trainer.train(resume_from_checkpoint=False)

Epoch,Training Loss,Validation Loss,Auc
1,5.727800,0.395835,0.083732
2,0.389800,0.319866,0.556220
3,0.300300,0.225835,0.952153
4,0.149300,0.009119,1.000000
5,0.079800,0.329565,1.000000


TrainOutput(global_step=75, training_loss=1.3294019158681234, metrics={'train_runtime': 144.6715, 'train_samples_per_second': 12.096, 'train_steps_per_second': 1.037, 'total_flos': 7403051037947904.0, 'train_loss': 1.3294019158681234, 'epoch': 5.0})

### Evaluate

#### Get the validation and test responses for each state

We get the agent's responses for the validation and test conversations. We use the validation responses for backward-induction threshold-tuning (our scalable alternative to grid search).

In [18]:
val_prompts = list(optimal_state_action_pairs["val"]["state"].values)
test_prompts = list(optimal_state_action_pairs["test"]["state"].values)

print("Getting test responses...")

responses_test = []
logprobs_test = []
batch_size = 72 # change to suit hardware

for i in tqdm(range(0, len(test_prompts), batch_size)):
    batch_prompts = test_prompts[i:i+batch_size]
    batch = tokenizer(batch_prompts, 
                      return_tensors="pt", 
                      padding=True, 
                      add_special_tokens=True,
                      truncation=True).to("cuda")

    with torch.no_grad():
        outputs = trainer.model.generate(
            **batch, 
            max_new_tokens=2, 
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id,
            temperature=None, top_p=None, top_k=None,
            return_dict_in_generate=True, output_scores=True
            # greedy decoding: so output_scores = output_logits
        )

    seqs = outputs.sequences
    prompt_len = batch['input_ids'].shape[1]   # Length of the input prompts

    # Slice to get only the generated new tokens
    generated_tokens = seqs[:, prompt_len:]
    decoded_outputs = tokenizer.batch_decode(generated_tokens,
                                             skip_special_tokens=True)
    decoded_outputs = [d.strip().lower() for d in decoded_outputs]

    responses_test.extend(decoded_outputs)

    scores = outputs.scores
    logprobs = [torch.log_softmax(s, dim=-1) for s in scores]
    logprobs = logprobs[0][torch.arange(logprobs[0].size(0)), seqs[:, -2].view(-1)]
    logprobs_test.extend(logprobs.cpu().numpy())

print("Getting validation responses...")

responses_val = []
logprobs_val = []
batch_size = 72

for i in tqdm(range(0, len(val_prompts), batch_size)):
    batch_prompts = val_prompts[i:i+batch_size]
    batch = tokenizer(batch_prompts, 
                      return_tensors="pt", 
                      padding=True, 
                      truncation=True).to("cuda")

    with torch.no_grad():
        outputs = trainer.model.generate(
            **batch, 
            max_new_tokens=2, 
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id,
            temperature=None, top_p=None, top_k=None,
            return_dict_in_generate=True, output_scores=True
        )

    seqs = outputs.sequences
    prompt_len = batch['input_ids'].shape[1]   # Length of the input prompts

    # Slice to get only the generated new tokens
    generated_tokens = seqs[:, prompt_len:]
    decoded_outputs = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
    decoded_outputs = [d.strip().lower() for d in decoded_outputs]

    responses_val.extend(decoded_outputs)

    scores = outputs.scores
    logprobs = [torch.log_softmax(s, dim=-1) for s in scores]
    logprobs = logprobs[0][torch.arange(logprobs[0].size(0)), seqs[:, -2].view(-1)]
    logprobs_val.extend(logprobs.cpu().numpy())

Getting test responses...


  0%|          | 0/2 [00:00<?, ?it/s]

Getting validation responses...


  0%|          | 0/1 [00:00<?, ?it/s]

#### Store validation and test responses

In [20]:
predictions = optimal_state_action_pairs["test"].copy()
predictions["response"] = responses_test
predictions["logprob"] = logprobs_test
predictions["prob"] = predictions["logprob"].apply(lambda x: math.exp(x))
predictions.loc[predictions["response"]=="yes", "prob_yes"] =\
    predictions.loc[predictions["response"]=="yes", "prob"]
predictions.loc[predictions["response"]!="yes", "prob_yes"] =\
    1.0 - predictions.loc[predictions["response"]!="yes", "prob"]

predictions_val = optimal_state_action_pairs["val"].copy()
predictions_val["response"] = responses_val
predictions_val["logprob"] = logprobs_val
predictions_val["prob"] =\
    predictions_val["logprob"].apply(lambda x: math.exp(x))
predictions_val.loc[predictions_val["response"]=="yes", "prob_yes"] =\
    predictions_val.loc[predictions_val["response"]=="yes", "prob"]
predictions_val.loc[predictions_val["response"]!="yes", "prob_yes"] =\
    1.0 - predictions_val.loc[predictions_val["response"]!="yes", "prob"]

test_with_predictions  = pd.merge(left=data_transcripts["test"],
                                     right=predictions[["conversation_id",
                                                        "state", "prob_yes"]],
                                     left_on=["conversation_id", "s" + str(m1)],
                                     right_on=["conversation_id", "state"],
                                     how="left", validate="one_to_one")\
                                        .drop(columns=["state"])\
                                        .rename(columns={"prob_yes": "prob_yes_" + str(m1)})\
                              .merge(right=predictions[["conversation_id",
                                                        "state", "prob_yes"]],
                                     left_on=["conversation_id", "s" + str(m2)],
                                     right_on=["conversation_id", "state"],
                                     how="left", validate="one_to_one")\
                                        .drop(columns=["state"])\
                                        .rename(columns={"prob_yes": "prob_yes_" + str(m2)})\

test_with_predictions.loc[
    test_with_predictions["prob_yes_" + str(m1)].isnull(),
                          "prob_yes_" + str(m1)] = 0
test_with_predictions.loc[
    test_with_predictions["prob_yes_" + str(m2)].isnull(),
                          "prob_yes_" + str(m2)] = 0

val_with_predictions  = pd.merge(left=data_transcripts["val"],
                                    right=predictions_val[["conversation_id",
                                         "state", "prob_yes"]],
                                    left_on=["conversation_id", "s" + str(m1)],
                                    right_on=["conversation_id", "state"],
                                    how="left", validate="one_to_one")\
                                        .drop(columns=["state"])\
                                        .rename(columns={"prob_yes": "prob_yes_" + str(m1)})\
                             .merge(right=predictions_val[["conversation_id", 
                                         "state", "prob_yes"]],
                                    left_on=["conversation_id", "s" + str(m2)],
                                    right_on=["conversation_id", "state"],
                                    how="left", validate="one_to_one")\
                                        .drop(columns=["state"])\
                                        .rename(columns={"prob_yes": "prob_yes_" + str(m2)})\

val_with_predictions.loc[
    val_with_predictions["prob_yes_" + str(m1)].isnull(),
                         "prob_yes_" + str(m1)] = 0
val_with_predictions.loc[
    val_with_predictions["prob_yes_" + str(m2)].isnull(),
                         "prob_yes_" + str(m2)] = 0

print("Val", m1, "ROC-AUC: {:.2f}".format(
    roc_auc_score(val_with_predictions["is_sale"].values,
                  val_with_predictions["prob_yes_" + str(m1)].values)))
print("Test", m1, "ROC-AUC: {:.2f}".format(
    roc_auc_score(test_with_predictions["is_sale"].values,
                  test_with_predictions["prob_yes_" + str(m1)].values)))

print("Val", m2, "ROC-AUC: {:.2f}".format(
    roc_auc_score(val_with_predictions["is_sale"].values,
                  val_with_predictions["prob_yes_" + str(m2)].values)))
print("Test", m2, "ROC-AUC: {:.2f}".format(
    roc_auc_score(test_with_predictions["is_sale"].values,
                  test_with_predictions["prob_yes_" + str(m2)].values)))

Val 45 ROC-AUC: 1.00
Test 45 ROC-AUC: 0.99
Val 60 ROC-AUC: 1.00
Test 60 ROC-AUC: 0.98


#### Get optimal thresholds using backward-induction threshold tuning

We could do a grid search, but this is much faster.

In [56]:
def simulate_threshold(threshold_m1, threshold_m2, df):
    # quit at m1
    calls_quit_at_m1 = df.loc[(df["prob_yes_" + str(m1)] < threshold_m1)]
    
    # continue at m1, ended before m2
    calls_continued_at_m1_and_ended = df.loc[(df["prob_yes_" + str(m1)] >= threshold_m1) &
                                             (df["duration"]<m2)]
    
    # continued at m1, did not end before m2, quit at m2
    calls_continued_at_m1_and_quit_at_m2 = df.loc[(df["prob_yes_" + str(m1)] >= threshold_m1) &
                                                  (df["prob_yes_" + str(m2)] < threshold_m2) &
                                                  (df["duration"]>=m2)]
    
    # continue at m1, did not end before m2, continued at m2
    calls_continued_at_m2 = df.loc[(df["prob_yes_" + str(m1)] >= threshold_m1) &
                                   (df["prob_yes_" + str(m2)] >= threshold_m2) &
                                   (df["duration"]>=m2)]
    
    assert len(calls_quit_at_m1) + len(calls_continued_at_m1_and_ended) +\
          len(calls_continued_at_m1_and_quit_at_m2) + \
          len(calls_continued_at_m2) == len(df)

    total_sales = calls_continued_at_m1_and_ended["is_sale"].sum() +\
                    calls_continued_at_m2["is_sale"].sum()
    total_sales_benefit = total_sales * BENEFIT_PER_POSITIVE_OUTCOME

    total_time = (len(calls_quit_at_m1) * m1 +\
                  len(calls_continued_at_m1_and_quit_at_m2) * m2 +\
                    calls_continued_at_m1_and_ended["duration"].sum() +\
                    calls_continued_at_m2["duration"].sum())
    total_cost = total_time * COST_PER_UNIT_TIME
    
    total_reward = (total_sales_benefit - total_cost)
    average_reward = total_reward/len(df)

    # assert benefit.sum() == total_sales_benefit
    # assert np.isclose(cost.sum(), total_cost)
    # assert np.isclose(reward.sum(), total_reward)

    return total_reward, average_reward, total_sales, total_time

print("Test set reward without the stopping agent:")
total_reward_noagent, average_reward_noagent,\
    total_sales_noagent, total_time_noagent =\
    simulate_threshold(0, 0, test_with_predictions)
print("Total reward on test:", total_reward_noagent)
print("Avg. reward on test:", average_reward_noagent)
print("Total sales on test:", total_sales_noagent)
print("Total time on test (seconds):", total_time_noagent)

assert int(total_sales_noagent) == int(test_with_predictions["is_sale"].sum())
assert total_time_noagent == test_with_predictions["duration"].sum()
assert total_reward_noagent ==\
    (total_sales_noagent * BENEFIT_PER_POSITIVE_OUTCOME - total_time_noagent * COST_PER_UNIT_TIME)
assert average_reward_noagent ==\
    total_reward_noagent / len(test_with_predictions)

Test set reward without the stopping agent:
Total reward on test: -77.97900000000004
Avg. reward on test: -1.5595800000000009
Total sales on test: 25
Total time on test (seconds): 3279.79


In [61]:
best_threshold_at_m = {}
num_grid_points = 10000

m = m1
prob_column = "prob_yes_" + str(m)    
best_reward = -10000000
for candidate_threshold in np.linspace(val_with_predictions[prob_column].min()-10**-12,
                                       val_with_predictions[prob_column].max()+10**-12,
                                       num=num_grid_points):
    
    total_reward, average_reward, total_sales, total_time =\
        simulate_threshold(0, candidate_threshold, val_with_predictions)
    
    if average_reward > best_reward:
        best_reward = average_reward
        best_threshold_at_m[m] = candidate_threshold

print("Best threshold at m=" + str(m) + ":", best_threshold_at_m[m])

m = m2
prob_column = "prob_yes_" + str(m)    
best_reward = -10000000
for candidate_threshold in np.linspace(val_with_predictions[prob_column].min()-10**-12,
                                       val_with_predictions[prob_column].max()+10**-12,
                                       num=num_grid_points):
    
    total_reward, average_reward, total_sales, total_time =\
        simulate_threshold(candidate_threshold, best_threshold_at_m[m1], val_with_predictions)
    
    if average_reward > best_reward:
        best_reward = average_reward
        best_threshold_at_m[m] = candidate_threshold

print("Best threshold at m=" + str(m) + ":", best_threshold_at_m[m])

total_reward_agent, average_reward_agent, \
    total_sales_agent, total_time_agent = simulate_threshold(best_threshold_at_m[m1],
                                                             best_threshold_at_m[m2],
                                                             test_with_predictions)

print()
print("Test set reward with the stopping agent:")
print("Total reward on test:", total_reward_agent)
print("Avg. reward on test:", average_reward_agent)
print("Total sales on test:", total_sales_agent)
print("Total time on test:", total_time_agent)
print()

print("Comparative results:")
print("Sales lost by stopping agent:", total_sales_noagent - total_sales_agent)
print("Time saved by stopping agent (seconds):", total_time_noagent - total_time_agent)
print("Time saved by stopping agent (%):", (total_time_noagent - total_time_agent)/\
                                            total_time_noagent * 100)

Best threshold at m=45: 0.00010000464725449688
Best threshold at m=60: 0.0014000817639441545

Test set reward with the stopping agent:
Total reward on test: -60.261000000000024
Avg. reward on test: -1.2052200000000004
Total sales on test: 24
Total time on test: 3002.61

Comparative results:
Sales lost by stopping agent: 1
Time saved by stopping agent (seconds): 277.17999999999984
Time saved by stopping agent (%): 8.451150835876682
